# Complete guide

## Introduction

This Notebook contains an overview of the basic functionality of the simulator. It introduces the simplest ways to get started with the simulator, and it dives into more advanced concepts that will allow you to get a sense of the flexibility of the system. At the end of this guide, you should be able to configure the pre-loaded simulations with custom parameters and metrics.

## Main components
The following diagram provides a birds-eye view of the simulation dynamics. (Note that the diagram does not include content creators, which are optional.)
<img src="figures/diagram.jpg" width=500>

A simulation needs the following components:

- **Users**: agents who interact with each other and with items.
- **Model**: agent that defines the behavior of the sociotechnical system. The model mediates the interactions among users and between users and the system.
- **Items**: passive components that are served to the users by the model. (Items may come from a fixed catalog or may be dynamically generated by **creators**.)
- **Measurements**: modules built into the models which automatically compute information about the system.

## Dynamics
The following steps are at the heart of the simulations:
1. The **model** presents the **users** with some recommended **items**. The recommendations are generated in accordance with the specific recommender system algorithm the model is using (e.g., content filtering, popularity, etc.). The input to the algorithm is based on the model's _prediction_ or _knowledge_ of user preferences.
2. The **users** view the items presented by the **model**, and interact with some **items** according to some _actual_ preferences.
3. The **model** updates its system state (such as the prediction of user preferences) based on the interactions of **users** with **items**, and it takes some **measurements**.

We will see that this framework is very flexible and it can be a generalization of many classic and new models.

## Quick start: instantiate a model and run
The fastest way to get started is to choose a model, instantiate it with no parameters, and run it for some time steps. Here we run a simple [content filtering recommendation system](https://elucherini.github.io/t-recs/reference/models.html#module-models.content) (please refer to the [BaseRecommender documentation](https://elucherini.github.io/t-recs/reference/models.html#module-models.recommender) for a complete list of class attributes and methods shared by all models, as that information is currently incomplete in the docs of the other pre-loaded models).

Content filters infer information about the _attributes_ of users based on their past interactions and recommend items with similar attributes to those of users.

In [1]:
import pandas as pd
import numpy as np

import trecs
from trecs.models import ContentFiltering
from trecs.random import Generator
from trecs.metrics import MSEMeasurement, InteractionSpread, RecSimilarity, AverageFeatureScoreRange

In [15]:
# Create ContentFiltering instance without arguments
default_filtering = ContentFiltering()
# add an MSE measurement
default_filtering.add_metrics(MSEMeasurement())
# Run for 5 time steps
default_filtering.run(timesteps=5, repeated_items=False)

100%|██████████| 5/5 [01:14<00:00, 14.81s/it]


In [57]:
default_filtering.interactions

array([ 721,  350, 1232, 1229,  681,  694,  254,   83,  239,  160,  926,
       1101, 1140,  527,  666,  435,   45,  441, 1217,  544,  682,  673,
        634,  880, 1140,  715, 1136,  594,  960,  833, 1234,   97,  328,
        124, 1164, 1228,  594,  248, 1010,  230,  435,  931,  812, 1096,
        479,  255, 1078,   84,  192,  249,  198,  420,  953,  423,  364,
        773,  230,  124,  307, 1200,   75,  592,  122,  437,  319,  560,
        441,  372,   21, 1222,  960,   17,  660,   93,  229, 1114,  825,
       1166, 1191,  682,  487,  237,  499,  452,  846, 1008, 1162,  766,
        110, 1163,  566,  985,  152,  809,  112,  496,  698,  687,  895,
        146])

In [74]:

print(default_filtering.items_shown[0])

[2.31501513 1.61273792 1.23986727 ... 2.35904469 1.46501122 2.31102203]
[441 925 238 985 721 399 104 491 593 545]


In [77]:

k=5
shown_item_scores = np.take(default_filtering.predicted_scores.value, default_filtering.items_shown)
item_val = dict(zip(default_filtering.items_shown[0], np.round(shown_item_scores[0],2)))
shown_item_ranks=np.argsort(shown_item_scores, axis=1)
top_k_items = np.take(default_filtering.items_shown, shown_item_ranks[:,k:])


#print(shown_item_scores[0])
print(shown_item_ranks[0])

#top_k_indices=np.argsort(shown_item_ranks, axis=1)[:,k:]

#top_k = np.take(default_filtering.items_shown, top_k_indices)

#top_n_indices = np.argwhere(shown_item_ranks < 5)
#print(top_k[0])
#default_filtering
print(top_k_items[0])
print(item_val)

[9 6 7 8 5 3 1 2 0 4]
[985 925 238 441 721]
{441: 3.72, 925: 3.46, 238: 3.62, 985: 3.38, 721: 4.39, 399: 3.15, 104: 2.72, 491: 3.07, 593: 3.09, 545: 2.65}


In [67]:
print(default_filtering.items_shown[0])

for ix, n in enumerate(default_filtering.predicted_scores.value[0]):
    print(ix, round(n,2))
#print(np.round(default_filtering.predicted_scores.value[0],1))

[441 925 238 985 721 399 104 491 593 545]
0 2.32
1 1.61
2 1.24
3 2.92
4 2.58
5 2.16
6 2.61
7 2.34
8 2.28
9 2.21
10 2.24
11 2.31
12 2.34
13 2.42
14 1.42
15 2.77
16 2.64
17 2.54
18 1.61
19 2.07
20 2.96
21 2.02
22 2.54
23 2.32
24 1.68
25 1.66
26 2.1
27 1.25
28 2.07
29 2.21
30 1.39
31 1.78
32 1.96
33 2.22
34 2.84
35 2.2
36 1.76
37 1.64
38 1.57
39 1.78
40 1.8
41 1.75
42 2.82
43 1.3
44 1.63
45 1.6
46 2.27
47 2.09
48 2.42
49 2.21
50 2.41
51 2.53
52 1.87
53 2.03
54 1.42
55 1.83
56 2.33
57 1.78
58 2.07
59 2.59
60 2.38
61 2.57
62 1.64
63 1.63
64 3.27
65 2.39
66 2.38
67 1.69
68 2.81
69 1.91
70 1.42
71 1.5
72 2.11
73 2.06
74 1.88
75 2.08
76 0.95
77 1.25
78 2.86
79 2.83
80 2.61
81 2.68
82 2.36
83 1.89
84 2.3
85 1.74
86 1.69
87 2.31
88 1.89
89 2.49
90 1.92
91 2.46
92 2.51
93 2.6
94 2.18
95 2.1
96 2.82
97 2.58
98 1.08
99 2.45
100 2.76
101 1.7
102 2.33
103 1.42
104 2.72
105 2.37
106 3.4
107 2.32
108 1.73
109 2.72
110 2.61
111 2.33
112 0.9
113 1.44
114 1.98
115 1.88
116 1.96
117 2.38
118 1.15
119 2.32


1027 2.28
1028 1.78
1029 1.44
1030 1.82
1031 2.11
1032 2.83
1033 2.41
1034 1.49
1035 2.21
1036 1.62
1037 2.42
1038 2.03
1039 2.09
1040 2.25
1041 2.4
1042 1.63
1043 1.9
1044 2.97
1045 2.18
1046 1.96
1047 2.15
1048 1.01
1049 0.73
1050 1.96
1051 2.21
1052 2.43
1053 2.39
1054 3.0
1055 1.6
1056 1.85
1057 1.53
1058 1.77
1059 1.88
1060 2.44
1061 1.71
1062 1.94
1063 2.37
1064 2.1
1065 2.11
1066 2.14
1067 2.24
1068 2.85
1069 2.76
1070 2.14
1071 2.77
1072 2.15
1073 2.55
1074 1.7
1075 2.27
1076 2.64
1077 1.49
1078 2.07
1079 1.85
1080 2.22
1081 3.22
1082 1.5
1083 1.25
1084 2.4
1085 2.01
1086 2.07
1087 2.01
1088 1.25
1089 2.08
1090 2.78
1091 2.04
1092 1.79
1093 2.94
1094 2.11
1095 1.66
1096 1.94
1097 2.04
1098 2.24
1099 2.44
1100 1.68
1101 2.05
1102 0.73
1103 1.85
1104 2.3
1105 2.9
1106 2.27
1107 2.22
1108 1.34
1109 2.84
1110 1.69
1111 1.85
1112 2.41
1113 2.68
1114 2.4
1115 2.44
1116 2.58
1117 1.4
1118 2.47
1119 2.12
1120 1.75
1121 2.49
1122 1.29
1123 1.97
1124 2.33
1125 1.22
1126 2.37
1127 0.91
11

In [35]:
#default_filtering.indices == -1
#k=np.where(default_filtering.indices ==-1)
# k=default_filtering.indices == -1
# k.shape
default_filtering.items_shown

x=np.isin(default_filtering.interactions, default_filtering.items_shown)

#default_filtering.items_shown[0,:]
indices = np.where(np.in1d(default_filtering.interactions, default_filtering.items_shown))[0]
indices


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [25]:
# Collect measurements about the simulation
results = default_filtering.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

Results of the simulation:


,mse,timesteps
0,0.469807,0
1,125.074007,1
2,147.454217,2
3,147.302546,3
4,145.550000,4
5,143.962536,5


In what follows, we expand on this minimal example to gain a deeper understanding of what happens under the hood.

## Models

As in the ``Quick Start``, if you want to run a simulation, the smallest piece of information you need is the model you want to run. There are a number of pre-loaded models that work out of the box. We continue to use a generic content filtering recommendation system; please see the docs for a [list of already-implemented models](https://elucherini.github.io/t-recs/reference/models.html#).

Recall that content filters infer information about the _attributes_ of users based on their past interactions and recommend items with similar attributes to those of users.

In [26]:
# Again, we instantiate the model with no arguments
default_filtering = ContentFiltering()

In the cell above, we instantiated a content filtering recommender system with default parameters. We print below the default number of users and items in the system.

In [27]:
print(f"Number of users in system: {default_filtering.num_users}")
print(f"Number of items in system: {default_filtering.num_items}")

Number of users in system: 100
Number of items in system: 1250


The model also created a representation for both users and items.

In [28]:
print("In content filtering, the default model representation of users and items are given by:")
print(f"- An all-zeros matrix of users of dimension {default_filtering.predicted_user_profiles.shape}")
print(f"- A randomly generated matrix of items of dimension {default_filtering.predicted_item_attributes.shape}")

In content filtering, the default model representation of users and items are given by:
- An all-zeros matrix of users of dimension (100, 1000)
- A randomly generated matrix of items of dimension (1000, 1250)


Formally, content filtering supports user profiles of size `|num_users x num_attributes|` and item attributes of size `|num_attributes x num_items|`.

### Set number of users or items
We can customize the number of users and items in the system:

In [29]:
# instantiate with a different number of items and users
number_of_items = 5000
number_of_users = 500
filtering = ContentFiltering(num_items = number_of_items, num_users=number_of_users)
print(f"The number of items in the system is now {filtering.num_items}.")
print(f"The number of users in the system is now {filtering.num_users}.")

The number of items in the system is now 5000.
The number of users in the system is now 500.


Note that the representations of items and users are set accordingly:

In [30]:
print(f"The size of item_attributes is {filtering.predicted_item_attributes.shape}.")
print(f"The size of user_profiles is {filtering.predicted_user_profiles.shape}.")

The size of item_attributes is (1000, 5000).
The size of user_profiles is (500, 1000).


## User predictions and Items
We might also want to define our own representation of users and items. We can do so by defining matrices that satisfy the constraints of the model. The constraints for ContentFiltering (some of which have been mentioned above) are:

- User profiles must be of size `|num_users x num_attributes|`.
- User profiles are matrices of integers representing the number of interactions of each user with items that have a given attributes. `user_profiles[i, j]` represents the number of interactions user `i` had with items with attribute `j`.
- Item attributes must be of size `|num_attributes x num_items|`.
- The model doesn't define any constraint on item attributes. If `item_attributes` is binary, then its `[i, j]`th element is 1 if item `j` is attributed attribute `i`; otherwise, it's 0. Item attributes can also be real-valued, representing the probability that each attribute has to describe items.

If you're already familiar with Numpy: the model is compatible with `ndarray` and _array_like_ data structures. 

If you're not familiar with Numpy: the framework provides a random number generator that lets you draw from several distributions (which, in practice, is a thin wrapper around `numpy.random.Generator`). Please refer to the Numpy documentation for a [list of distributions](https://numpy.org/doc/stable/reference/random/generator.html?highlight=generator#distributions).

In [31]:
# Keep the dimensions small for easy visualization
number_of_users = 5
number_of_attributes = 10
number_of_items = 15
# We define user_representation using the standard integer generator in Numpy.
# We assume a number of interactions with each attribute in the interval [0,4).
user_representation = np.random.randint(4, size=(number_of_users, number_of_attributes))

# We define item_representation using the Generator that comes with the framework
# We assume a binary matrix with a binomial distribution

item_representation = Generator().binomial(
    n=1, p=.3, size=(number_of_attributes, number_of_items)
)
# Note that this is equivalent to:
# item_representation = np.random.Generator(np.random.MT19937()).binomial(n=1, p=.5, size=(...))

print(f"User representation (num_users x num_attributes):\n{str(user_representation)}\n")
print(f"Item representation (num_attributes x num_items):\n{str(item_representation)}")

User representation (num_users x num_attributes):
[[1 2 0 2 0 0 2 0 0 0]
 [2 2 1 1 2 1 2 3 2 3]
 [0 3 3 3 0 0 3 0 3 0]
 [3 3 2 2 3 2 0 0 1 2]
 [2 0 1 1 2 3 2 1 2 2]]

Item representation (num_attributes x num_items):
[[0 0 0 0 0 0 1 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 1 1 0 1 0 0]
 [0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 1 1 1 0 0 0 0 1]
 [1 0 0 0 1 1 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 1 1 0 0]
 [1 0 0 0 0 1 0 0 0 1 1 1 0 0 0]
 [1 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0 1 0 0]]


In [32]:
# Initialize with custom representations
filtering = ContentFiltering(user_representation=user_representation,
                             item_representation=item_representation)

# Check if they're equivalent
is_user_equivalent = "yes" if np.array_equal(user_representation, filtering.predicted_user_profiles) else "no"
is_item_equivalent = "yes" if np.array_equal(item_representation, filtering.predicted_item_attributes) else "no"
print("Is user_profiles equivalent to user_representation? %s." % is_user_equivalent)
print("Is item_attributes equivalent to item_representation? %s." % is_item_equivalent)

Is user_profiles equivalent to user_representation? yes.
Is item_attributes equivalent to item_representation? yes.


You can also initialize models with `user_representation` and `item_representation` individually. In this case, the representation that has not been initialized will adapt to the size defined by the user.

In [33]:
# Let's only initialize user_profiles
filtering = ContentFiltering(user_representation=user_representation)
print("After initializing user_profiles, the size of item_attributes (and so the number of attributes in the system) adapts automatically to it.")
print(f"Size of user_profiles, as defined above: {str(filtering.predicted_user_profiles.shape)}.")
print(f"Size of item_attributes: {str(filtering.predicted_item_attributes.shape)}.\n")

# The same happens by only initializing item_attributes
filtering = ContentFiltering(item_representation=item_representation)
print("After initializing item_attributes, the size of user_profiles (and so the number of attributes in the system) adapts automatically to it.")
print(f"Size of item_attributes, as defined above: {str(filtering.predicted_item_attributes.shape)}.")
print(f"Size of user_profiles: {str(filtering.predicted_user_profiles.shape)}.")

After initializing user_profiles, the size of item_attributes (and so the number of attributes in the system) adapts automatically to it.
Size of user_profiles, as defined above: (5, 10).
Size of item_attributes: (10, 1250).

After initializing item_attributes, the size of user_profiles (and so the number of attributes in the system) adapts automatically to it.
Size of item_attributes, as defined above: (10, 15).
Size of user_profiles: (100, 10).


## Run a simulation
We can run a simulation for the predefined number of time steps (50), or define our own duration.

In [34]:
# let's initialize a model with both user_representation and item_representation defined above
filtering = ContentFiltering(user_representation=user_representation,
                            item_representation=item_representation)
filtering.add_metrics(MSEMeasurement()) # add MSE Measurement
# Run the model for the predefined number of timesteps:
filtering.run()

100%|██████████| 50/50 [00:00<00:00, 2720.36it/s]


At the end of the simulation, we can examine the results of the measurements. For example:

In [35]:
# To get the measurements of all timesteps<=50
measurements = filtering.get_measurements()

# Measurements can be easily converted to pandas DataFrame objects

pd.DataFrame(measurements).head()

,mse,timesteps
0,0.741495,0
1,0.740219,1
2,0.714323,2
3,0.683384,3
4,0.655116,4


## Measurements
At each time step of the simulation, measurement modules calculate a quantity based on the system state. An example of such quantity is the mean squared error between the predicted user profiles and the actual user profiles -- that is, how close is the model to predicting the real preferences of the system?

It's easy to define new metrics, but in this guide we will use some of the pre-loaded metrics to get a better sense of how they work. For a list of pre-loaded metrics and their descriptions, see the [docs](https://elucherini.github.io/t-recs/reference/metrics.html).

### View metrics

The metrics monitored are stored in the the model's `metrics` attribute, which contains a list of objects of type [Measurement](https://elucherini.github.io/t-recs/reference/metrics.html#measurement-base-class).

In [36]:
# The metrics tracked by each model can be examined by printing the `metrics` list.
print("The system is currently monitoring these metrics:")
print(filtering.metrics)

The system is currently monitoring these metrics:


### Add metrics
To **maintain compatibility with pandas**, we suggest to **only add metrics to instances of models that have not been run yet**. This is to avoid having measurements that start at different time steps, resulting in arrays of different length. Feel free to disregard this advice if pandas compatibility is not important to your application.

We can instantiate a model, add a new metric, and then run the model.

We will add InteractionSpread, which provides a measure of the homogeneity of user interactions in the system as a whole.



In [37]:
#change the number of items and users to make metric values more reasonable and dimensions distinguishable
number_of_items=100
number_of_users=50
number_of_attributes=20

#change the distribution of item attributes so average feature score range can be added later
item_representation = Generator().normal(size=(number_of_attributes, number_of_items))
filtering = ContentFiltering(num_users=number_of_users, num_items=number_of_items, 
                             num_attributes=number_of_attributes,
                             item_representation=item_representation,
                             record_base_state=True)

# This method accepts a variable number of metrics
filtering.add_metrics(MSEMeasurement(), InteractionSpread())

print("These are the current metrics:")
print(filtering.metrics)

These are the current metrics:
[<trecs.metrics.measurement.MSEMeasurement object at 0x7fcbf9956210>, <trecs.metrics.measurement.InteractionSpread object at 0x7fcbde070a50>]


We will also add [RecSimilarity](https://elucherini.github.io/t-recs/reference/metrics.html#recsimilarity), which measures the similarity between interaction patterns for pairs of users. In order to measure Jaccard similarity, we must specify which pairs of users we want to compare.

In [38]:
js_pairs = [(u1_idx, u2_idx) for u1_idx in range(filtering.num_users) for u2_idx in range(filtering.num_users) if u1_idx != u2_idx] 
filtering.add_metrics(RecSimilarity(pairs=js_pairs))

print("These are the current metrics:")
print(filtering.metrics)

These are the current metrics:
[<trecs.metrics.measurement.MSEMeasurement object at 0x7fcbf9956210>, <trecs.metrics.measurement.InteractionSpread object at 0x7fcbde070a50>, <trecs.metrics.measurement.RecSimilarity object at 0x7fcbf9bc0990>]


Now we will add average feature score range, a metric for evaluating within list diversity based on the range of item attribute values

In [39]:
filtering.add_metrics(AverageFeatureScoreRange())

In [40]:
# now we run the model
filtering.run(timesteps=5)
measurements = filtering.get_measurements()
pd.DataFrame(measurements)

100%|██████████| 5/5 [00:00<00:00, 53.54it/s]


,mse,interaction_spread,rec_similarity,afsr,timesteps
0,0.935492,None,None,None,0
1,1.456278,-49,0.056505479384737066,5.320404285881352,1
2,1.265972,0.5,0.07021272669468823,5.801227750597289,2
3,1.175250,0,0.07018198681053331,5.807356060456102,3
4,1.142149,0,0.0702012800142582,5.725780952521848,4
5,1.128374,0,0.0700213946805218,5.706819689308626,5


Measurements at time step 0 can be undefined (`None`, `NaN`, etc.) because it denotes the measurements before the start of the simulation. MSE is defined at the beginning because the system is initialized with random predictions of the user profiles; in contrast, homogeneity is meaningless before the simulation begins because there are no user interactions to consider.

## System state


Some applications might require keeping a history of the system's internal state for future processing. This is useful, for example, to study the evolution of predicted user profiles. The framework provides an interface to store and access all the states of each component over time through the `SystemStateModule` interface. Some components are tracked by default, others are added into the individual models.

In [41]:
# note that we instantiated filtering with the option record_base_state=True
system_state = filtering.get_system_state()
print("These are the system state components being monitored:")
print(system_state.keys())

These are the system state components being monitored:
dict_keys(['predicted_users', 'actual_user_scores', 'predicted_items', 'predicted_user_scores', 'timesteps'])


In [42]:
print("There are as many states as the timesteps for which we ran the system + the initial state.")
print("For example, the history of predicted_user_profiles has length:", (len(system_state['predicted_users'])))
# the last states correspond to the current state of the components
print("Furthermore, the last state is in the history of a component corresponds to its current state.")
print("Is this true for predicted_user_profiles?", np.array_equal(system_state['predicted_users'][5], filtering.predicted_user_profiles))

There are as many states as the timesteps for which we ran the system + the initial state.
For example, the history of predicted_user_profiles has length: 6
Furthermore, the last state is in the history of a component corresponds to its current state.
Is this true for predicted_user_profiles? True


To start tracking a new component, you can use the [add_state_variable()](https://elucherini.github.io/t-recs/reference/components.html#base.base_components.SystemStateModule.add_state_variable). Note that state variables can only be monitored if they must inherit from the `BaseComponent` class. Creating new state variables is outside of the scope of this guide, so please refer to the [advanced-models](advanced-models.ipynb) and the [advanced-metrics](advanced-metrics.ipynb) notebooks.

## "Real" users, "real" items
Most of what we've seen so far about users refers to the predictions that the system makes about users' preferences and the item attributes. This framework allows for modeling system predictions as well as "real" users and items. These "real" user and item attributes can be passed into the simulation model; while they remain invisible by the recommender system algorithm, they dictate which items users interact with, and this interaction data is what the recommender system uses to continually retrain.

By default, the preference ordering over items for a particular user is defined by the dot product between the item profile and the user profile. You can

In [43]:
# consider a toy example where all users have a very strong preference for items with the attribute at index 0,
# and only the first item has the attribute at index 0

# each row is a user
real_users = np.array([
    [1, 0, 0, 0, 0.7],
    [1, 0, 0.1, 0, 0],
    [1, 0, 0, 0.2, 0],
    [1, 0, 0.5, 0, 0],
])

# each column is an item
real_items = np.array([
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.1, 0.0, 0.2, 0.0],
    [0.0, 0.0, 0.2, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.3],
    [0.1, 0.1, 0.2, 0.3, 0.0],
])

# instantiate content filtering model; note that the system's internal model of
# each user's preferences over items will essentially be random
# we set the number of items shown to users at each iteration to be 5
filtering = ContentFiltering(actual_user_representation=real_users,
                             actual_item_representation=real_items,
                             num_items_per_iter=5)

In [44]:
# Now when we run the model, we can look at the most recent recommendations
# and the items the users actually interacted with
filtering.run(timesteps=1)
print("Items shown to each user during the first timestep (row = one user)")
print(filtering.items_shown)

print()
print("Items each user chose to interact with (item at index i represents the item user i chose)")
print(filtering.interactions)

print()
filtering.run(timesteps=1)
print("Items shown to each user during the second timestep (row = one user).")
print("Note that the the first item shown is item 0, indicating the recommender system has learned the users' preferences.")
print(filtering.items_shown)

/mnt/c/Users/matth/pton-research/recsys/t-recs/trecs/models/recommender.py:662: UserWarning: No measurements are currently defined for the simulation. Please add measurements if desired.
  warnings.warn(error_msg)
100%|██████████| 1/1 [00:00<00:00, 442.76it/s]

Items shown to each user during the first timestep (row = one user)
[[2 0 1 3 4]
 [1 3 4 0 2]
 [0 1 2 3 4]
 [0 2 1 3 4]]

Items each user chose to interact with (item at index i represents the item user i chose)
[0 0 0 0]

Items shown to each user during the second timestep (row = one user).
Note that the the first item shown is item 0, indicating the recommender system has learned the users' preferences.
[[0 2 1 3 4]
 [0 2 1 3 4]
 [0 2 1 3 4]
 [0 2 1 3 4]]


### User interactions with items
The Users class also determines how users interact with items. The default behavior is defined in [get_user_feedback()](https://elucherini.github.io/t-recs/reference/components.html#components.users.Users.get_user_feedback). In short, when the system presents items to users, users internally evaluate the items and choose the one item that maximizes the dot product between their own preferences and the item attributes. This default behavior can be overriden to provide a custom model of user-item interaction (for an example, see [DNUsers](https://elucherini.github.io/t-recs/reference/components.html#components.users.DNUsers)).

## Model parameters about user interactions
Models also provide a few initialization parameters that can be used to tweak the behavior of the model in regards to user interactions. Specifically, models determine the number of items to present users at each iteraction through parameter `num_items_per_iter`. The default is 10 items per user per iteration.

### Other model parameters
Please refer to [the docs](https://elucherini.github.io/algo-segregation/reference/models.html).